# 데이터 로더 만들기
- 학습모델에 입력으로 넣기 위한 data loader
- 일기준 데이터 로더
  - Binary label(1/0) return: load_data_il_B(종목코드)
  - Ternary label(U/H/D) return: load_data_il(종목코드)
- 분기준 데이터 로더:
  - Binary label(1/0) return:load_data_bun_B(종목코드)
  - Binary label(U/H/D) return:load_data_bun(종목코드)

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
#!pip install JPype1
#!pip install konlpy
#!pip install pyMySQL
#!pip install -U finance-datareader

In [ ]:
#-- Import packages --
import os
import numpy as np
import pandas as pd
#import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
#import konlpy
#from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')
#-- Import packages :END --

In [ ]:
######################################################################

In [ ]:
#-------------------------------------------------------------------------
#-- load_data_il_B() -----------------------------------------------------
# 일기준 데이터 로더: Binary 1/0 (U/D) 가져오기
def load_data_il_B(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_il_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # 레이블 'H' 는 빼기, Up/Down만 남기기
  il_data_df = il_data_df[il_data_df['label'] != 'H']
  # 레이블 숫자로 변환 return
  il_data_df['label'] = il_data_df['label'].replace({'U': 1, 'D': 0})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_il_B() : END--------------------------------------------

#---------------------------------------------------------------------
#-- load_data_il() ---------------------------------------------------
# 일기준 데이터 로더: Ternary U/H/D 가져오기
def load_data_il(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_il_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # 레이블 숫자로 변환 return
  #il_data_df['label'] = il_data_df['label'].replace({'U': 3, 'H': 2, 'D': 1})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_il() :End-----------------------------------------------

#-- load_data_bun_B() ------------------------------------------------
# 분기준 데이터 로더: Binary 1/0 (U/D) 가져오기
def load_data_bun_B(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
   # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_bun_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # Null 있는 행 삭제
  il_data_df = il_data_df.dropna()
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # 레이블 'H' 는 빼기, Up/Down만 남기기
  il_data_df = il_data_df[il_data_df['label'] != 'H']
  # 레이블 숫자로 변환 return
  il_data_df['label'] = il_data_df['label'].replace({'U': 1, 'D': 0})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_bun_B() :End--------------------------------------------

#-- load_data_bun() --------------------------------------------------
# 분기준 데이터 로더: Ternary U/H/D 가져오기
def load_data_bun(stock_code):
  # 실행확인을 위한 화면 출력
  #print(stock_code)
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM {stock_code}_bun_dataset x"
  # DB 검색결과를 dataframe에 저장
  il_data_df = pd.read_sql_query(sql, conn)
  # 컬럼 이름 정리
  il_data_df.columns= ['text','label']
  # Null 있는 행 삭제
  il_data_df = il_data_df.dropna()
  # 레이블 숫자로 변환 return
  il_data_df['label'] = il_data_df['label'].replace({'U': 3, 'H': 2, 'D': 1})
  # DB close
  conn.close()
  # df 반환
  return il_data_df
#-- load_data_bun() :End----------------------------------------------


In [ ]:
#=====================================================================
# 메인 함수 실행 테스트
#---------------------------------------------------------------------
data_df = load_data_il_B('034950')
data_df.head(3)
#=====================================================================